In [1]:
import numpy as np
import logging
import json

from IonDiff import analyze_descriptors  as AD_library

from datetime import datetime

Analysis of atomistic descriptors extracted from the diffusive paths (under active development).

At the input folder, a XDATCAR file with all the configurations of the system through simulation is required.
Optionally, a POSCAR can be supplied with the initial configuration.
As well, an INCAR specifying POTIM (simulation time step) and NBLOCK (number of simulation steps between
consecutive configurations in the XDATCAR) is necessary.

In [2]:
class args:
    def __init__(self):
        args.MD_path = '.'  # Path to the input molecular dynamics simulation files
        args.reference_path = None  # Path to a folder with a stoichiometric POSCAR structure file for the simulation of the given material

# Initialize class
args = args()

In [3]:
# Configuring loggin information
logging.basicConfig(
    filename=f'../logs/analyze_descriptors_{datetime.now().strftime("%Y-%m-%d_%H:%M:%S")}.log',
    format='%(asctime)s - [%(levelname)s] - %(message)s',
    level=logging.INFO,
)

# Saving logging information
logging.info(f'Task: Analysing atomistic descriptors from MD simulations database at {args.MD_path}.')

# Calling the library and loading the class
inp = AD_library.descriptors(args)

# Computing descriptors
time_interval      = inp.time_until_diffusion()
temporal_duration  = inp.duration_of_diffusion()
spatial_length     = inp.length_of_diffusion(outer='nan')
n_diffusive_events = inp.n_diffusive_events()
residence_time     = inp.residence_time(args)[0] if args.reference_path is not None else None

# Save descriptors as dictionary
descriptors = {
    'MD_path':      args.MD_path,
    'delta_t_min':  np.min(time_interval),
    'delta_t_max':  np.max(time_interval),
    'delta_t_mean': np.mean(time_interval),
    'delta_r_min':  np.min(temporal_duration),
    'delta_r_max':  np.max(temporal_duration),
    'delta_r_mean': np.mean(temporal_duration),
    'gamma':        residence_time
}

# Logging update
logging.info(f'Descriptors successfully extracted.')

# Write the dictionary to the file in JSON format
with open(f'{args.MD_path}/atomistic_descriptors.json', 'w') as json_file:
    json.dump(descriptors, json_file)

# Logging update
logging.info(f'Descriptors successfully saved.')